# 🏦 Credit Risk Scorecard — Notebook 1: Exploratory Data Analysis (EDA) & Data Cleaning

**Project:** Credit Risk Scorecard + IFRS 9 Expected Credit Loss (ECL) Engine  
**Dataset:** Give Me Some Credit — Kaggle Competition Data  
**Objective:** Predict the probability that a borrower will experience serious financial distress within the next two years (foundation for the PD model)

**Author:** *Your Name*  
**Date:** *2026*

---

### 📋 Notebook Contents
1. Load data & inspect structure
2. Target variable analysis (default rate)
3. Handle missing values
4. Handle outliers
5. Feature distribution visualisation
6. Correlation analysis
7. Save cleaned dataset

## Step 0: Upload Data File (Google Colab Only)

If running in **Google Colab**, uncomment and run the cell below to upload `cs-training.csv`.  
If running locally, make sure `cs-training.csv` is in the same directory as this notebook.

In [ ]:
# Uncomment the lines below if running in Google Colab
# from google.colab import files
# uploaded = files.upload()  # Select cs-training.csv from your local machine

## Step 1: Import Libraries & Load Data

In [ ]:
# ── Core libraries ──────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

# Plot styling
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['font.size'] = 12
sns.set_style('whitegrid')

print('All libraries imported successfully.')

In [ ]:
# ── Load raw training data ───────────────────────────────────
df = pd.read_csv('cs-training.csv', index_col=0)

print(f'Dataset loaded successfully.')
print(f'Shape: {df.shape[0]:,} rows  x  {df.shape[1]} columns')
print()
df.head()

In [ ]:
# ── Rename columns for readability ───────────────────────────
df.columns = [
    'default',           # Target: serious delinquency in next 2 years (1=yes, 0=no)
    'revolving_util',    # Revolving credit utilisation rate
    'age',               # Borrower age
    'late_30_59',        # Number of times 30-59 days past due
    'debt_ratio',        # Monthly debt payments / monthly gross income
    'monthly_income',    # Monthly gross income
    'open_credit_lines', # Number of open credit lines and loans
    'late_90',           # Number of times 90+ days past due
    'real_estate_loans', # Number of real estate loans or lines
    'late_60_89',        # Number of times 60-89 days past due
    'dependents'         # Number of dependents
]

print('Columns renamed:')
for col in df.columns:
    print(f'  · {col}')

## Step 2: Data Overview

In [ ]:
# ── Data types, missing values, unique counts ────────────────
info_df = pd.DataFrame({
    'dtype': df.dtypes,
    'missing_count': df.isnull().sum(),
    'missing_pct': (df.isnull().sum() / len(df) * 100).round(2).astype(str) + '%',
    'unique_values': df.nunique()
})

print('=== Data Overview ===')
print(info_df)
print()
print(f'Columns with missing values: {df.isnull().any().sum()}')

In [ ]:
# ── Descriptive statistics ───────────────────────────────────
print('=== Descriptive Statistics ===')
df.describe().round(2)

## Step 3: Target Variable Analysis

In [ ]:
# ── Default rate summary ─────────────────────────────────────
default_counts = df['default'].value_counts()
default_rate = df['default'].mean() * 100

print('=== Target Variable: default ===')
print(f'Non-default (0): {default_counts[0]:,}  ({100 - default_rate:.1f}%)')
print(f'Default     (1): {default_counts[1]:,}  ({default_rate:.1f}%)')
print(f'\nOverall default rate : {default_rate:.2f}%')
print(f'Class imbalance ratio: ~1:{int(default_counts[0]/default_counts[1])}  (default : non-default)')

# ── Plot ─────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
colors = ['#2ecc71', '#e74c3c']

# Bar chart
axes[0].bar(['Non-Default (0)', 'Default (1)'], default_counts.values,
            color=colors, edgecolor='white', linewidth=1.5)
axes[0].set_title('Default Distribution (Count)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Number of Borrowers')
for i, v in enumerate(default_counts.values):
    axes[0].text(i, v + 500, f'{v:,}', ha='center', fontweight='bold')

# Pie chart
axes[1].pie(
    default_counts.values,
    labels=[f'Non-Default\n{100-default_rate:.1f}%', f'Default\n{default_rate:.1f}%'],
    colors=colors, startangle=90,
    wedgeprops={'edgecolor': 'white', 'linewidth': 2}
)
axes[1].set_title('Default Rate', fontsize=14, fontweight='bold')

plt.suptitle('Target Variable Distribution', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.savefig('plot_01_target_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: plot_01_target_distribution.png')

## Step 4: Handle Missing Values

In [ ]:
# ── Visualise missing values ─────────────────────────────────
missing = df.isnull().sum()
missing = missing[missing > 0]

plt.figure(figsize=(8, 4))
bars = plt.bar(missing.index, missing.values / len(df) * 100,
               color='#e74c3c', alpha=0.8)
plt.title('Missing Value Rate by Column', fontsize=14, fontweight='bold')
plt.ylabel('Missing Rate (%)')
plt.xticks(rotation=15)
for bar in bars:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
             f'{bar.get_height():.1f}%', ha='center', fontweight='bold')
plt.tight_layout()
plt.savefig('plot_02_missing_values.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: plot_02_missing_values.png')

In [ ]:
# ── Impute missing values with median ────────────────────────
# Median is preferred over mean for skewed distributions (e.g. income)

income_median = df['monthly_income'].median()
df['monthly_income'].fillna(income_median, inplace=True)
print(f'monthly_income: imputed with median = {income_median:,.0f}')

dep_median = df['dependents'].median()
df['dependents'].fillna(dep_median, inplace=True)
print(f'dependents    : imputed with median = {dep_median:.0f}')

print()
print(f'Remaining missing values: {df.isnull().sum().sum()}')

## Step 5: Handle Outliers

In [ ]:
# ── Outlier detection & treatment ────────────────────────────
print('=== Outlier Treatment ===')

# 1. Age — remove implausible values
print(f'age < 18 : {(df["age"] < 18).sum()} records')
print(f'age > 100: {(df["age"] > 100).sum()} records')
before = len(df)
df = df[(df['age'] >= 18) & (df['age'] <= 100)]
print(f'  → Removed {before - len(df)} records with implausible age\n')

# 2. revolving_util — clip to [0, 1] (utilisation rate should be a proportion)
print(f'revolving_util > 1: {(df["revolving_util"] > 1).sum()} records')
df['revolving_util'] = df['revolving_util'].clip(0, 1)
print(f'  → Clipped to [0, 1]\n')

# 3. Late payment counts — values of 96/98 are likely data entry codes, not real counts
for col in ['late_30_59', 'late_60_89', 'late_90']:
    n = (df[col] >= 96).sum()
    print(f'{col} >= 96: {n} records (likely miscoded)')
    df[col] = df[col].clip(0, 10)
    print(f'  → Clipped to [0, 10]')

print()
print(f'Final dataset shape: {df.shape[0]:,} rows x {df.shape[1]} columns')

## Step 6: Feature Distribution Visualisation

In [ ]:
# ── Distribution plots by default status ─────────────────────
features = [col for col in df.columns if col != 'default']

fig, axes = plt.subplots(3, 4, figsize=(18, 12))
axes = axes.flatten()

for i, col in enumerate(features):
    for label, color, name in [(0, '#2ecc71', 'Non-Default'), (1, '#e74c3c', 'Default')]:
        data = df[df['default'] == label][col]
        axes[i].hist(data, bins=30, alpha=0.6, color=color, label=name, density=True)
    axes[i].set_title(col, fontsize=11, fontweight='bold')
    axes[i].legend(fontsize=8)

# Hide unused subplots
for j in range(len(features), len(axes)):
    axes[j].set_visible(False)

plt.suptitle('Feature Distributions by Default Status\n(Green = Non-Default, Red = Default)',
             fontsize=15, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('plot_03_feature_distributions.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: plot_03_feature_distributions.png')

## Step 7: Correlation Analysis

In [ ]:
# ── Correlation heatmap ──────────────────────────────────────
corr = df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))  # Show lower triangle only

plt.figure(figsize=(12, 9))
sns.heatmap(
    corr, mask=mask,
    annot=True, fmt='.2f', cmap='RdYlGn',
    center=0, vmin=-1, vmax=1,
    square=True, linewidths=0.5,
    cbar_kws={'shrink': 0.8}
)
plt.title('Feature Correlation Heatmap', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('plot_04_correlation_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

# Print correlation with target variable
print('=== Correlation with Target Variable (default) ===')
target_corr = corr['default'].drop('default').sort_values(key=abs, ascending=False)
for col, val in target_corr.items():
    direction = '(+)' if val > 0 else '(-)'
    print(f'  {direction}  {col:25s}  {val:+.4f}')

## Step 8: Default Rate by Age Group

In [ ]:
# ── Default rate by age band ─────────────────────────────────
df['age_group'] = pd.cut(df['age'],
                          bins=[18, 25, 35, 45, 55, 65, 100],
                          labels=['18-25', '26-35', '36-45', '46-55', '56-65', '65+'])

age_default = df.groupby('age_group')['default'].mean() * 100
avg_rate = age_default.mean()

plt.figure(figsize=(10, 5))
colors = ['#e74c3c' if v > avg_rate else '#3498db' for v in age_default.values]
bars = plt.bar(age_default.index.astype(str), age_default.values,
               color=colors, edgecolor='white', linewidth=1.5)
plt.axhline(y=avg_rate, color='black', linestyle='--', alpha=0.7,
            label=f'Average default rate: {avg_rate:.1f}%')
plt.title('Default Rate by Age Group', fontsize=14, fontweight='bold')
plt.xlabel('Age Group')
plt.ylabel('Default Rate (%)')
plt.legend()
for bar in bars:
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             f'{bar.get_height():.1f}%', ha='center', fontweight='bold')
plt.tight_layout()
plt.savefig('plot_05_age_default_rate.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: plot_05_age_default_rate.png')

## Step 9: Save Cleaned Dataset

In [ ]:
# ── Export cleaned data ──────────────────────────────────────
df_clean = df.drop(columns=['age_group'])
df_clean.to_csv('cs-training-clean.csv', index=False)

print('Cleaned dataset saved as: cs-training-clean.csv')
print(f'Final shape : {df_clean.shape[0]:,} rows x {df_clean.shape[1]} columns')
print(f'Missing values: {df_clean.isnull().sum().sum()}')
print()
print('=== EDA Summary ===')
print(f'  · Raw data         : 150,000 rows')
print(f'  · Cleaned data     : {df_clean.shape[0]:,} rows')
print(f'  · Overall default rate : {df_clean["default"].mean()*100:.2f}%')
print(f'  · Missing value treatment : median imputation (monthly_income, dependents)')
print(f'  · Outlier treatment       : age filter, revolving_util clip, late payment clip')
print()
print('Next step → notebook_02_pd_model.ipynb: Logistic Regression PD Model')

---
## 📁 Output Files

| File | Description |
|------|-------------|
| `cs-training-clean.csv` | Cleaned training dataset |
| `plot_01_target_distribution.png` | Default rate distribution |
| `plot_02_missing_values.png` | Missing value analysis |
| `plot_03_feature_distributions.png` | Feature distributions by default status |
| `plot_04_correlation_heatmap.png` | Correlation heatmap |
| `plot_05_age_default_rate.png` | Default rate by age group |

---
**Next Notebook:** `notebook_02_pd_model.ipynb` — Logistic Regression PD Model (AUC, KS, Calibration)